## Imports

In [93]:
import os
import dlib
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
import scipy
import pandas as pd
import random
import math

## Preprocessing:

In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../shape_predictor_68_face_landmarks.dat")

In [7]:
def write_cropped_image(orig_path, new_path=None):
    print("Processing file: {}".format(orig_path))
    img = dlib.load_rgb_image(orig_path)
    face_area = detector(img, 1)
    shape = predictor(img, face_area[0])
    pts = np.array( # the 27 points on the face
        [[shape.part(0).x, shape.part(0).y], [shape.part(3).x, shape.part(3).y],
         [shape.part(4).x, shape.part(4).y], [shape.part(5).x, shape.part(5).y], 
         [shape.part(6).x, shape.part(6).y], [shape.part(7).x, shape.part(7).y],
         [shape.part(8).x, shape.part(8).y], [shape.part(9).x, shape.part(9).y],
         [shape.part(10).x, shape.part(10).y], [shape.part(11).x, shape.part(11).y], 
         [shape.part(12).x, shape.part(12).y], [shape.part(13).x, shape.part(13).y],
         [shape.part(14).x, shape.part(14).y], [shape.part(15).x, shape.part(15).y],
         [shape.part(16).x, shape.part(16).y], [shape.part(26).x, shape.part(26).y], 
         [shape.part(25).x, shape.part(25).y], [shape.part(24).x, shape.part(24).y],
         [shape.part(23).x, shape.part(23).y], [shape.part(22).x, shape.part(22).y],
         [shape.part(27).x, shape.part(27).y], [shape.part(21).x, shape.part(21).y], 
         [shape.part(20).x, shape.part(20).y], [shape.part(19).x, shape.part(19).y],
         [shape.part(18).x, shape.part(18).y], [shape.part(17).x, shape.part(17).y]], 
         dtype=np.int32)
    # Create polygon shaped mask
    mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.int32)
    cv2.fillPoly(mask, np.int32([pts]), 1)
    mask = mask.astype(bool)
    # Fill in polygon with image
    out = np.zeros_like(img)
    out[mask] = img[mask]

    #src = cv2.imread(f, 0)
    cropped_image = out[int(face_area[0].top()):int(face_area[0].bottom()), int(face_area[0].left()):int(face_area[0].right())]
    cropped_image = cv2.resize(cropped_image, (130, 130))
    
    #plt.imshow(cropped_image)
    #plt.show()
    cv2.imwrite(new_path, cropped_image)


In [ ]:
write_cropped_image("../Ck+/anger/S503_001_00000071.png", "a")

In [ ]:
# Given a folder
def preprocess_folder_of_emotions(folder_path, emotions_list, pic_type="*.png"):
    write_to = folder_path + "_new"
    for emotion in emotions_list:
        #print("first for loop")
        #print(pic_type)
        emo_folder_path = folder_path + "/" + emotion
        write_path = write_to + "/" + emotion
        #print("elp_0")
        for pic_file in glob.glob(os.path.join(emo_folder_path, pic_type)):
            #print("elp")
            #print("Writing file: {}".format(pic_file))
            write_pic_path = write_path + pic_file[-22:]
            write_cropped_image(pic_file, write_pic_path)

                                  
emotions_list = ["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]
preprocess_folder_of_emotions("../CK+", emotions_list, "*.png")

## LBP Feature Vectors

In [9]:
# LBP
emotions_list = ["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]
list_of_vecs = []
counter = 0
for a in emotions_list:
    faces_folder_path = "../CK+_new/" + str(a)
    for pic_file in glob.glob(os.path.join(faces_folder_path, "*.png")):
        src = cv2.imread(pic_file, 0)
        lbp_image = local_binary_pattern(src, 8, 2, method='nri_uniform')
        histogram = scipy.stats.itemfreq(lbp_image)
        vector = []
        for c, b in histogram:
            vector.append(b)
        
        list_of_vecs.append([counter, a, vector])
        counter +=1
#print(len(list_of_vecs[0]))

/var/folders/d6/73t68jg509l9qqjp_k_6st940000gp/T/ipykernel_48744/3920418892.py:10: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  histogram = scipy.stats.itemfreq(lbp_image)


## ORB Vectors

In [10]:
counter = 0
for a in emotions_list:
    faces_folder_path = "../CK+_new/" + a
    for f in glob.glob(os.path.join(faces_folder_path, "*.png")):
        #print("Processing file: {}".format(f))
        src = cv2.imread(f, 0)
        src = src[1:129, 1:129]
        #print(src.shape)
        #print(len(src[0]))
        #print('hi')
        # Initiate ORB detector
        orb = cv2.ORB_create()
        # find the keypoints with ORB
        kp = orb.detect(src, None)
        # compute the descriptors with ORB
        kp, des = orb.compute(src, kp)
        #print(des[0])
        #print(des[0].shape)
        #print(des[0].tolist())
        list_of_vecs[counter].append(des[0].tolist())
        counter += 1

In [37]:
print(len(list_of_vecs[0]))

4


## Post-Processing

In [34]:
def z_fold_thingy(full_list):
    overall_list = []
    for val in full_list:
        vector_LBP = val[2]
        vector_ORB = val[3]
        max_lbp = max(vector_LBP)
        vector_LBP = [a/max_lbp for a in vector_LBP]

        max_orb = max(vector_ORB)
        vector_ORB = [a / max_orb for a in vector_ORB]
        overall_list.append([val[1], vector_LBP+vector_ORB])

    c = 1e-5
    for counter, (emo, vector) in enumerate(overall_list):
        avg = sum(vector)/len(vector)
        r = 0
        for a in vector:
            r += (a-avg)**2
        
        new = []
        for counter, a in enumerate(vector):
            new.append(100 * (a-avg)/(r+c))
        overall_list[counter][1] = new
    return overall_list

In [52]:
print(len(z_fold_thingy(list_of_vecs)[0][1]))

91


In [87]:
def train_svm(train_percent, trials, list_of_vecs):
    emo_dict = {"anger": 0, "contempt": 1, "disgust": 2, "fear": 3, "happiness": 4, "neutral": 5, "sadness": 6,
            "surprise": 7}
    shuffle_desk = [a for a in list_of_vecs]
    
    #random.shuffle(shuffle_desk)
    #print(shuffle_desk[0])
    #print(len(shuffle_desk))
    train_count = int((len(list_of_vecs) * (train_percent/100))//1) # Round down
    print(train_count)
    
    for trial_number in range(trials):
        train_set = random.sample(range(len(list_of_vecs)), train_count)
        train_labels, train_vectors = list(zip(*[shuffle_desk[a] for a in train_set]))
        
        train_vectors = np.matrix(train_vectors, dtype=np.float32)
        train_labels = np.array([emo_dict[a] for a in train_labels], dtype=int)

        test_set = [a for a in range(len(list_of_vecs)) if a not in train_set]
        test_labels, test_vectors = list(zip(*[shuffle_desk[a] for a in test_set]))
        
        test_vectors = np.matrix(test_vectors, dtype=np.float32)
        test_labels = np.array([emo_dict[a] for a in test_labels], dtype=int)
        
        # Repeat until every emotion is in test_labels
        if 0 not in test_labels or 1 not in test_labels or 2 not in test_labels or 3 not in test_labels or 4 not in test_labels or 5 not in test_labels or 6 not in test_labels or 7 not in test_labels:
            print("hi")
            while 0 not in test_labels or 1 not in test_labels or 2 not in test_labels or 3 not in test_labels or 4 not in test_labels or 5 not in test_labels or 6 not in test_labels or 7 not in test_labels:
                train_set = random.sample(range(len(list_of_vecs)), train_count)
                train_labels, train_vectors = list(zip(*[shuffle_desk[a] for a in train_set]))

                train_vectors = np.matrix(train_vectors, dtype=np.float32)
                train_labels = np.array([emo_dict[a] for a in train_labels], dtype=int)


                test_set = [a for a in range(len(list_of_vecs)) if a not in train_set]
                test_labels, test_vectors = list(zip(*[shuffle_desk[a] for a in test_set]))

                test_vectors = np.matrix(test_vectors, dtype=np.float32)
                test_labels = np.array([emo_dict[a] for a in test_labels], dtype=int)
                print(len(train_set))
                print(len(test_set))
        
        # Train the SVM
        svm = cv2.ml.SVM_create()
        svm.setType(cv2.ml.SVM_C_SVC)
        
        svm.setKernel(cv2.ml.SVM_RBF)
        svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))
        print(train_vectors)
        print(train_labels)
        svm.train(train_vectors, cv2.ml.ROW_SAMPLE, train_labels)
        
        # Test it
        # Confusion matrix: column headers = correct, rows = what was outputted
        confusion_matrix_count = np.zeros((8,8))
        right = 0
        wrong = 0
        print(len(test_labels))
        for label, vector in zip(train_labels, train_vectors):
            
            response = svm.predict(vector)[1]
            confusion_matrix_count[int(response)][label] += 1
            if response == label:
                right += 1
            else:
                wrong += 1
        print(right/(right + wrong) * 100)
        print(confusion_matrix_count)
        
        

        
    

In [ ]:
thingy = z_fold_thingy(list_of_vecs)
train_svm(95, 1, thingy)

In [56]:
print(random.sample(range(len(list_of_vecs)), 5))
    

[571, 466, 210, 625, 848]


In [129]:
def train_svm_2(train_percent, trials, list_of_vecs):
    emo_dict = {"anger": 0, "contempt": 1, "disgust": 2, "fear": 3, "happiness": 4, "neutral": 5, "sadness": 6,
            "surprise": 7}
    shuffle_desk = [a for a in list_of_vecs]
    
    random.shuffle(shuffle_desk) # Shuffles desk
    
    past_emo = 0
    list_of_emotion_of_vectors = {} # Creates a dictionary
    for [emo,vector] in shuffle_desk:
        if emo not in list_of_emotion_of_vectors.keys():
            list_of_emotion_of_vectors[emo] = [vector]
        else:
            list_of_emotion_of_vectors[emo].append(vector)
    print()
          
    #print(shuffle_desk[0])
    #print(len(shuffle_desk))
    #train_count = int((len(list_of_vecs) * (train_percent/100))//1) 
    #print(train_count)
    
    for a in range(trials):
        train_vectors = []
        train_labels = []
        test_vectors = []
        test_labels = []
        for emotion in list_of_emotion_of_vectors.keys():
            length = len(list_of_emotion_of_vectors[emotion])
            print(length)
            chosen = int(math.ceil(length * (train_percent/100))) # round down

            for vector in list_of_emotion_of_vectors[emotion][:chosen]:
                train_vectors.append(vector)
                train_labels.append(emo_dict[emotion])
            for vector in list_of_emotion_of_vectors[emotion][chosen:]:
                test_vectors.append(vector)
                test_labels.append(emo_dict[emotion])
        print(test_labels[0])
    
         
        # Train the SVM
        svm = cv2.ml.SVM_create()
        svm.setType(cv2.ml.SVM_C_SVC)
        
        svm.setKernel(cv2.ml.SVM_RBF)
        svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))
        #print(train_vectors)
        #print(train_labels)
        svm.train(np.matrix(train_vectors, dtype=np.float32), cv2.ml.ROW_SAMPLE, np.array(train_labels, dtype=int))
        
        # Test it
        # Confusion matrix: column headers = correct, rows = what was outputted
        confusion_matrix_count = np.zeros((8,8))
        right = 0
        wrong = 0
        print(len(test_labels))
        for label, vector in zip(test_labels, np.matrix(test_vectors, dtype=np.float32)):
            
            response = svm.predict(vector)[1]
            confusion_matrix_count[int(response)][label] += 1
            if response == label:
                right += 1
            else:
                wrong += 1
        print(right/(right + wrong) * 100)
        print(confusion_matrix_count)

        


        
    

In [130]:
thingy = z_fold_thingy(list_of_vecs)
train_svm_2(95, 1, thingy)


593
83
25
18
69
59
28
45
5
42
69.04761904761905
[[ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 2.  0.  2.  1.  3. 29.  1.  4.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]]
